In [167]:
%load_ext autoreload
%autoreload 2

## Explicit Aspects evaluation

In [1]:
import os
os.chdir("..")

In [2]:
from Preprocessors.ReviewPreprocessor import ReviewPreprocessor
from Aspects.ExplicitAspectExtractor import ExplicitAspectExtractor
import re
import pandas as pd

In [3]:
tagged_hotels_corpus = open("data/tagged_hotels_corpus.txt", "r")

In [80]:
tagged_hotels_corpus.seek(3)
current_review_number = 1
review_text = []
temp_review_text = ""
for line in tagged_hotels_corpus.readlines():
    splited = line.split("###")
    sentence_text=splited[1]
    sentence_meta_data = splited[0]
    meta_data_items = sentence_meta_data.strip().split(" ")
    regex_groups = re.search(r'\[c([0-9]+)\]\[s([0-9]+)\]', meta_data_items[0]).groups()
    review_id = int(regex_groups[0])
    sentence_number = int(regex_groups[1])
    if current_review_number == review_id:
        temp_review_text += " " + sentence_text
    elif review_id == current_review_number+1:
        review_text.append(temp_review_text)
        temp_review_text = sentence_text
        current_review_number = review_id
# add add the last review
review_text.append(temp_review_text)

In [81]:
df = pd.DataFrame(review_text, columns = ["review"])

In [82]:
df

,review
0,this is quite the most delightful hotel i ha...
1,the actual hotel accomodations were very luxu...
2,"very good, but i am going back and won't stay..."
3,"we stayed here for one night, in december 201..."
4,the family that runs this hostel are wonderfu...
...,...
95,we arrived really late on friday evening and ...
96,bed and breakfast/ hostel style accommodation...
97,we stayed at cabanas del lago for one night f...
98,we found this place on the internet and staye...


### Preprocessing

In [84]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [112]:
df["cleaned_review"] = df["review"]
preprocessor = ReviewPreprocessor(df["cleaned_review"], nlp, subjectivity_threshold=0.4)
df["cleaned_review"] = preprocessor.start()

remove_tags done


100it [00:22,  4.42it/s]


spelling_correction done


100it [00:02, 37.23it/s]

remove_objective_sentences done


### Extraction des aspects explicits

In [122]:
explicit_aspects_extractor = ExplicitAspectExtractor(df["cleaned_review"], nlp)
explicit_10_aspects = explicit_aspects_extractor.start(10)
explicit_10_aspects_ = list(dict(explicit_10_aspects).keys())

In [123]:
explicit_10_aspects_

['room',
 'hotel',
 'breakfast',
 'view',
 'staff',
 'lake',
 'place',
 'service',
 'town',
 'restaurant']

#### Extract explicit aspects from dataset

In [ ]:
tagged_hotels_corpus.seek(3)
true_explicit_aspects = set()
for line in tagged_hotels_corpus.readlines():
    splited = line.split("###")
    
    aspects_metadata = splited[0].strip().split(" ", maxsplit=1)
    if len(aspects_metadata) > 1:
        aspects_metadata = aspects_metadata[1].split(",")
        for aspect in aspects_metadata:
            if "[p]" in aspect or "[u]" in aspect or aspect == "" :
                continue
            else:
                true_explicit_aspects.add(aspect[:-3].strip())

In [175]:
true_explicit_aspects

{'A/C',
 'accomodation',
 'air-conditioning',
 'amenity',
 'animal',
 'apartment',
 'aspect',
 'atmosphere',
 'attendant',
 'attention',
 'back room',
 'balcony',
 'bar',
 'bar area',
 'bath',
 'bathroom',
 'bathroom clanliness',
 'bathtub',
 'beach service',
 'bed',
 'bed comfort',
 'bedroom',
 'bike',
 'bike loan',
 'book exchange',
 'bookcase',
 'bread',
 'breakfast',
 'breakfast area',
 'breakfast selection',
 'broadband',
 'brunch',
 'building',
 'cabin',
 'cafe',
 'cafeteria',
 'cake',
 'carpet',
 'ceiling',
 'cereal',
 'chair',
 'cheese',
 'chef',
 'cleanliness',
 'closet',
 'coffee',
 'coffee maker',
 'coffee table',
 'common area',
 'common room',
 'computer',
 'computer room',
 'condiments',
 'cookware',
 'cost',
 'couch',
 'crockery',
 'croissant',
 'deck',
 'decor',
 'decoration',
 'dessert',
 'detergent',
 'dining',
 'dining area',
 'dining room',
 'dinner',
 'dinning',
 'dinning room',
 'distance',
 'dog',
 'doorman',
 'drawer',
 'driveway',
 'easy chair',
 'egg',
 'egg f

In [171]:
aspects_in_true = 0
for ie in explicit_10_aspects_:
    if ie in true_explicit_aspects:
        aspects_in_true += 1
aspects_in_true/len(true_explicit_aspects)

0.033962264150943396